---
title : "Overstromingsrisico scenarios"
execute:
    output: asis
---

Scenarios worden gebruikt om de mogelijke gevolgen van overstromingen in kaart te brengen. <br>
Hiermee kan de kans uit een fragilitycruve gecombineerd worden met de gevolgen die bepaald zijn bij een scenario om zo een beter inzicht te krijgen in het totale risico. <br>
Dit kan helpen om prioriteiten te stellen op basis van een totaal beeld en niet alleen op basis van kans of gevolg afzonderlijk. <br>
Om de scenarios te kunnen combineren moeten er een aantal stappen worden doorlopen:
- Omrekenen van dijkvakkansen naar trajectdeelkansen ([CalculateFloodScenarioProbability](#sec-CalculateFloodScenarioProbability))
- Belasting per deeltraject bepalen ([LoadFromFloodScenarioProbabilit](#sec-LoadFromFloodScenarioProbabilit))
- Met de belasting een bijpassende overstromingscenario selecteren ([SelectFloodScenarioFromLoad ](#sec-SelectFloodScenarioFromLoad))
- Van het scenario en de curve een geagregeerd risico bepalen ([CalculateFloodRisk](#sec-CalculateFloodRisk))


In [40]:
from pathlib import Path

from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter

In [41]:
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(config_path=path / "test_calculate_flood_scenario_probability.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [42]:
from toolbox_continu_inzicht.flood_scenarios import CalculateFloodScenarioProbability

## CalculateFloodScenarioProbability{sec-CalculateFloodScenarioProbability}

Dit voorbeeld laat zien hoe met behulp van de `CalculateFloodScenarioProbability`  dijkvakkansen naar trajectdeelkansen kunnen worden omgezet. <br>
Hiervoor is de gecombineerde faalkans per dijkvak nodig en een mapping van dijkvakken naar dijktraject delen van daar naar de overstromingsscenario gebieden.
```yaml|
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [-24,0,24,48]

DataAdapter:
    default_options:
        csv:
            sep: ","
    combined_failure_probability_data:
        type: csv
        file: "combined_failure_probability_data.csv"
    section_id_to_segment_id:
        type: csv
        file: "section_id_to_segment_id.csv"
    segment_failure_probability:
        type: csv
        file: "segment_failure_probability.csv"
```

In [43]:
calculate_flood_scenario_probability = CalculateFloodScenarioProbability(
    data_adapter=data_adapter
)
calculate_flood_scenario_probability.run(
    input=["combined_failure_probability_data", "section_id_to_segment_id"],
    output="segment_failure_probability",
)
# TODO: segment to dikesystem, only needed later on for risk = probability * damage

In [44]:
calculate_flood_scenario_probability.df_in_sections_to_segment

,section_id,segment_id,length
0,1,34003,2
1,3,34003,3
2,4,34003,1
3,5,34002,1
4,6,34002,1
5,7,34002,1
6,8,34002,1
7,9,34002,1
8,15,34002,2
9,16,34002,3


In [45]:
df = calculate_flood_scenario_probability.df_in_grouped_sections_failure_probability

In [46]:
df["section_id"][df["section_id"] < 1.5].min()

np.int64(1)

In [47]:
calculate_flood_scenario_probability.df_out

,segment_id,failure_probability
0,34003,0.183333
1,34002,0.000526


## LoadFromFloodScenarioProbability{sec-LoadFromFloodScenarioProbabilit}

Nu we de faalkansen per trajectdeel hebben kunnen we met de `LoadFromFloodScenarioProbability` de belasting per trajectdeel bepalen. <br>
Dit doen we om een bijpassende overstromingsscenario te kunnen selecteren. <br>
Met de input van de vorige stap `segment_failure_probability`, maken we een koppeling naar de bijhorende bres en fragilitycurve die representatief is voor een deeltraject met de tabel `breach_id_to_segment_id`. <br>
Deze `representative_section_id_fragilitycurve` kan vervolgens uit de data van de curves in `section_failure_probability_data` worden gehaald. <br>
Met de curve wordt de hydraulische belasting bepaald voor het deeltraject.

```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 48 ]

DataAdapter:
    default_options:
        csv:
            sep: ","
    segment_failure_probability:
        type: csv
        file: "segment_failure_probability.csv"
    breach_id_to_segment_id:
        type: csv
        file: "breach_id_to_segment_id.csv"
    section_failure_probability_data:
        type: csv
        file: "fragility_curve_multi_section.csv"
    flood_scenario_load_resultaten:
        type: csv
        path: "hidden_flood_scenario_load_resultaten.csv"

```

In [48]:
config = Config(config_path=path / "test_load_from_flood_scenario_probability.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [49]:
data_adapter.input("segment_failure_probability")

,Unnamed: 0,segment_id,failure_probability
0,0,34003,0.183333
1,1,34002,0.000526


In [50]:
data_adapter.input("breach_id_to_segment_id")

,segment_id,breach_id,representative_section_id_fragilitycurve
0,34003,1,1
1,34002,11,9


In [51]:
df = data_adapter.input("section_failure_probability_data")

In [52]:
from toolbox_continu_inzicht.flood_scenarios import LoadFromFloodScenarioProbability

In [53]:
load_from_flood_scenario_probability = LoadFromFloodScenarioProbability(
    data_adapter=data_adapter
)
load_from_flood_scenario_probability.run(
    input=[
        "segment_failure_probability",
        "breach_id_to_segment_id",
        "section_failure_probability_data",
    ],
    output="flood_scenario_load_resultaten",
)

In [54]:
data_adapter.input("flood_scenario_load_resultaten")

,segment_id,hydraulicload,breach_id
0,34003,14.95,1
1,34002,4.00,11


In [55]:
data_adapter.input("section_failure_probability_data")

,section_id,measure_id,hydraulicload,failure_probability
0,1,0,11.15,0.000000e+00
1,1,0,11.25,0.000000e+00
2,1,0,11.35,4.636286e-257
3,1,0,11.45,1.766899e-178
4,1,0,11.55,1.505861e-126
...,...,...,...,...
2243,12,3,19.60,1.000000e+00
2244,12,3,19.70,1.000000e+00
2245,12,3,19.80,1.000000e+00
2246,12,3,19.90,1.000000e+00


## SelectFloodScenarioFromLoad

```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 48 ]

DataAdapter:
    default_options:
        csv:
            sep: ","
    flood_scenario_loads:
        type: csv
        path: "flood_scenario_load_resultaten.csv"
    breach_location_metadata:
        type: csv
        path: "breach_location_metadata.csv"
    flood_scenario_grids:
        type: csv
        path: "flood_scenario_grids.csv"
```

In [56]:
config = Config(config_path=path / "test_select_flood_scenario_from_load.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [57]:
from toolbox_continu_inzicht.flood_scenarios import SelectFloodScenarioFromLoad

In [58]:
data_adapter.input("flood_scenario_loads")

,segment_id,hydraulicload,breach_id
0,34003,14.95,1
1,34002,4.00,11


In [59]:
data_adapter.input("breach_location_metadata")

,breach_id,hydaulicload_upperboundary,waterdepth_grid,casualties_grid,damage_grid,flooding_grid,affected_people_grid
0,11,2.335,NaN,NaN,NaN,NaN,NaN
1,11,2.497,scenario_4182_waterdiepte.tif,scenario_4182_slachtoffers.tif,scenario_4182_schade.tif,scenario_4182_nattevoeten.tif,scenario_4182_getroffenen.tif
2,11,2.723,scenario_5199_waterdiepte.tif,scenario_5199_slachtoffers.tif,scenario_5199_schade.tif,scenario_5199_nattevoeten.tif,scenario_5199_getroffenen.tif
3,11,2.930,scenario_5188_waterdiepte.tif,scenario_5188_slachtoffers.tif,scenario_5188_schade.tif,scenario_5188_nattevoeten.tif,scenario_5188_getroffenen.tif
4,1,2.344,NaN,NaN,NaN,NaN,NaN
5,1,2.506,scenario_5196_waterdiepte.tif,scenario_5196_slachtoffers.tif,scenario_5196_schade.tif,scenario_5196_nattevoeten.tif,scenario_5196_getroffenen.tif
6,1,2.726,scenario_5200_waterdiepte.tif,scenario_5200_slachtoffers.tif,scenario_5200_schade.tif,scenario_5200_nattevoeten.tif,scenario_5200_getroffenen.tif
7,1,2.948,scenario_5195_waterdiepte.tif,scenario_5195_slachtoffers.tif,scenario_5195_schade.tif,scenario_5195_nattevoeten.tif,scenario_5195_getroffenen.tif


In [60]:
load_from_flood_scenario_probability = SelectFloodScenarioFromLoad(
    data_adapter=data_adapter
)
load_from_flood_scenario_probability.run(
    input=[
        "flood_scenario_loads",
        "breach_location_metadata",
    ],
    output="flood_scenario_grids",
)

In [61]:
load_from_flood_scenario_probability.df_out

,segment_id,breach_id,hydaulicload_upperboundary,waterdepth_grid,casualties_grid,damage_grid,flooding_grid,affected_people_grid
7,34003,1,2.948,scenario_5195_waterdiepte.tif,scenario_5195_slachtoffers.tif,scenario_5195_schade.tif,scenario_5195_nattevoeten.tif,scenario_5195_getroffenen.tif
3,34002,11,2.930,scenario_5188_waterdiepte.tif,scenario_5188_slachtoffers.tif,scenario_5188_schade.tif,scenario_5188_nattevoeten.tif,scenario_5188_getroffenen.tif


### Keuze uit twee scenarios

Standaard wordt er per deeltraject één overstromingsscenario geselecteerd op basis van de hydraulische belasting. <br>
Met de parameter `return_two_scenarios` kan ervoor gekozen worden om twee overstromingsscenarios te selecteren. <br>
De gebruiker kan hiermee zelf bepalen welk scenario hij wil gebruiken voor de risico berekening. <br>
Of de gemiddelde schade van de twee scenario's gebruiken, of beide tonen. 

```yaml
GlobalVariables:
    rootdir: "data_sets/7.flood_scenarios"
    moments: [ -24, 0, 24, 48 ]
    SelectFloodScenarioFromLoad:
        return_two_scenarios: True

DataAdapter:
    default_options:
        csv:
            sep: ","
    flood_scenario_loads:
        type: csv
        path: "flood_scenario_load_resultaten.csv"
    breach_location_metadata:
        type: csv
        path: "breach_location_metadata.csv"
    flood_scenario_grids:
        type: csv
        path: "flood_scenario_grids.csv"
```

In [62]:
config = Config(
    config_path=path / "test_select_flood_scenario_from_load_two_scenarios.yaml"
)
config.lees_config()
data_adapter = DataAdapter(config=config)

In [63]:
load_from_flood_scenario_probability = SelectFloodScenarioFromLoad(
    data_adapter=data_adapter
)
load_from_flood_scenario_probability.run(
    input=[
        "flood_scenario_loads",
        "breach_location_metadata",
    ],
    output="flood_scenario_grids",
)

In [64]:
load_from_flood_scenario_probability.df_out

,segment_id,breach_id,hydaulicload_upperboundary,waterdepth_grid,casualties_grid,damage_grid,flooding_grid,affected_people_grid
5,34003,1,2.506,scenario_5196_waterdiepte.tif,scenario_5196_slachtoffers.tif,scenario_5196_schade.tif,scenario_5196_nattevoeten.tif,scenario_5196_getroffenen.tif
6,34003,1,2.726,scenario_5200_waterdiepte.tif,scenario_5200_slachtoffers.tif,scenario_5200_schade.tif,scenario_5200_nattevoeten.tif,scenario_5200_getroffenen.tif
1,34002,11,2.497,scenario_4182_waterdiepte.tif,scenario_4182_slachtoffers.tif,scenario_4182_schade.tif,scenario_4182_nattevoeten.tif,scenario_4182_getroffenen.tif
2,34002,11,2.723,scenario_5199_waterdiepte.tif,scenario_5199_slachtoffers.tif,scenario_5199_schade.tif,scenario_5199_nattevoeten.tif,scenario_5199_getroffenen.tif
